In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [ ]:
#Mecab install - 형태소 분석 tool
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-11-16 16:06:38--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::3403:4be7, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=a7qVjo7erKHqfddykW18lvqcbDE%3D&Expires=163707

사전 만들기


In [ ]:
import torch

NLP = torch.FloatTensor([0,1,0,0,0,0,0,1])
learning = torch.FloatTensor([0,0,0,1,0,0,0,1])

print(torch.cosine_similarity(NLP, learning, dim=0))

tensor(0.5000)


In [ ]:
from konlpy.tag import Mecab

m = Mecab()
res = m.pos("기계학습을 위한 텍스트 처리 방법을 공부해보자.")
print(res)

[('기계', 'NNG'), ('학습', 'NNG'), ('을', 'JKO'), ('위한', 'VV+ETM'), ('텍스트', 'NNG'), ('처리', 'NNG'), ('방법', 'NNG'), ('을', 'JKO'), ('공부', 'NNG'), ('해', 'XSV+EC'), ('보', 'VX'), ('자', 'EF'), ('.', 'SF')]


In [ ]:
from konlpy.tag import Mecab
from nltk import FreqDist
import numpy as np

def make_dic(texts):

  words = []
  m = Mecab()

  for snt in texts:
    res = m.pos(snt)
    for (lex,pos) in res: #pos는 품사(part of speech)
      if pos[0] == 'N' or pos[0] == 'V': # 명사 동사만 사전에 추가 // 'J' = 조사
        words.append(lex)

  # 단어의 빈도수 계산
  dic = FreqDist(np.hstack(words))
  print('사전 크기: {0:d}'.format(len(dic)))

  # 인덱스를 저장할 변수 초기화
  indexes = {}
  words = {}

  # 단어에 고유 번호(인덱스) 부여
  for num, word in enumerate(dic):
    idx = num+2
    indexes[word]= idx
    words[idx] = word

  # 문장 길이 정규화에 사용한 패딩 인덱스
  indexes['pad'] = 1
  words[1] = 'pad' #padding의 약자
  #  사전에 없는 단어에 대한 인덱스
  indexes['unk'] = 0
  words[0] = 'unk' #unknown의 약자
 
  return (indexes, words)

def word2index(indexes, word):
  idx = indexes[word] if word in indexes else indexes['unk']
  return idx

def index2word(words, index):
  w = words[index] if index in words else -1
  return w

texts = ["죽는 날까지 하늘을 우러러 한 점 부끄럼이 없기를",
         "잎새에 이는 바람에도 나는 괴로워 했다.",
         "별을 노래하는 마음으로 모든 죽어 가는 것을 사랑해야지",
         "그리고 나한테 주어진 길을 걸어가야겠다.",
         "오늘 밤에도 별이 바람에 스치운다."]

(indexes, words) = make_dic(texts)
 
print(word2index(indexes,'하늘'), word2index(indexes,'학교'))
print(index2word(words,4), index2word(words,0))

사전 크기: 24
4 0
하늘 unk


Word2Vec 사용하기

In [ ]:
from konlpy.tag import Mecab
import numpy as np
from gensim.models import Word2Vec, KeyedVectors

texts = ["죽는 날까지 하늘을 우러러 한 점 부끄럼이 없기를",
         "잎새에 이는 바람에도 나는 괴로워 했다.",
         "별을 노래하는 마음으로 모든 죽어 가는 것을 사랑해야지",
         "그리고 나한테 주어진 길을 걸어가야겠다.",
         "오늘 밤에도 별이 바람에 스치운다."]

# Word2Vec 학습에 사용할 데이터 만들기

m = Mecab()
result = []

for sent in texts:
  tag = m.pos(sent)
  words = []
  for (lex,pos) in tag:
    if pos[0] == 'N': #명사류만 추출
      words.append(lex)
  result.append(words)

print(result,"\n")

# Word2Vec 학습시키기
model = Word2Vec(sentences=result, size=10, window=1, min_count=1, workers=1, sg=0)
#size : 임베딩차원 window : 컨텍스트 윈도우 크기(앞에 1개, 뒤에 1개), min_count : 단어 최소 빈도 수 제한, workers : 학습을 위한 프로세스 수 sg : 0(CBOW) 1 (Skip-gram)

# 값 읽어오기
print(model.wv['하늘'])
# 유사한 단어 가져오기
print(model.wv.most_similar("하늘"),"\n")

# Word2Vec 모델 저장하기
model.wv.save_word2vec_format('/gdrive/My Drive/colab/text_rep/test_w2v')

# Word2Vec 모델 로드하기
loaded_model = KeyedVectors.load_word2vec_format("/gdrive/My Drive/colab/text_rep/test_w2v")

# 값 읽어오기
print(loaded_model.wv['하늘'])
# 유사한 단어 가져오기
print(loaded_model.wv.most_similar("하늘"))

[['날', '하늘', '점', '부끄럼'], ['잎새', '바람', '나'], ['별', '노래', '마음', '것', '사랑'], ['나', '길'], ['밤', '별', '바람']] 

[ 0.02746415 -0.0421521  -0.00413069 -0.01471152  0.0098496   0.01284836
  0.02050443  0.00189251 -0.03678587  0.0141834 ]
[('사랑', 0.7030139565467834), ('부끄럼', 0.3328326344490051), ('점', 0.27551624178886414), ('길', 0.2556310296058655), ('마음', 0.10330245643854141), ('잎새', 0.0970955416560173), ('밤', -0.05079442262649536), ('것', -0.05681941658258438), ('나', -0.060193076729774475), ('노래', -0.10640832781791687)] 

[ 0.02746415 -0.0421521  -0.00413069 -0.01471152  0.0098496   0.01284836
  0.02050443  0.00189251 -0.03678587  0.0141834 ]
[('사랑', 0.7030139565467834), ('부끄럼', 0.3328326344490051), ('점', 0.27551624178886414), ('길', 0.2556310296058655), ('마음', 0.10330245643854141), ('잎새', 0.0970955416560173), ('밤', -0.05079442262649536), ('것', -0.05681941658258438), ('나', -0.060193076729774475), ('노래', -0.10640832781791687)]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


glove 사용하기

In [ ]:
# colab 환경에 glove 설치
! pip install glove-python-binary

In [ ]:
from konlpy.tag import Mecab
import numpy as np
from glove import Corpus, Glove

texts = ["죽는 날까지 하늘을 우러러 한 점 부끄럼이 없기를",
         "잎새에 이는 바람에도 나는 괴로워 했다.",
         "별을 노래하는 마음으로 모든 죽어 가는 것을 사랑해야지",
         "그리고 나한테 주어진 길을 걸어가야겠다.",
         "오늘 밤에도 별이 바람에 스치운다."]

# GloVe 학습에 사용할 데이터 만들기

m = Mecab()
result = []

for sent in texts:
  tag = m.pos(sent)
  words = []
  for (lex,pos) in tag:
    if pos[0] == 'N':
      words.append(lex)
  result.append(words)

print(result,"\n")

# Co-Occurrence Matrix 생성
corpus = Corpus() 
corpus.fit(result, window=1)
#window = 1 : 앞, 뒤 하나씩 봄

# GloVe 학습시키기
glove = Glove(no_components=10, learning_rate=0.05) #10차원 임베딩
glove.fit(corpus.matrix, epochs=20, no_threads=1, verbose=True)
glove.add_dictionary(corpus.dictionary)

# 값 읽어오기
print(glove.word_vectors[glove.dictionary['하늘']])
# 유사한 단어 가져오기
print(glove.most_similar("하늘"),"\n")

# GloVe 모델 저장하기
glove.save('/gdrive/My Drive/colab/text_rep/test_glove')

# GloVe 모델 로드하기
loaded_model = glove.load("/gdrive/My Drive/colab/text_rep/test_glove")

# 값 읽어오기
print(loaded_model.word_vectors[glove.dictionary['하늘']])
# 유사한 단어 가져오기
print(loaded_model.most_similar("하늘"))

[['날', '하늘', '점', '부끄럼'], ['잎새', '바람', '나'], ['별', '노래', '마음', '것', '사랑'], ['나', '길'], ['밤', '별', '바람']] 

Performing 20 training epochs with 1 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
[-0.01958986  0.02647205  0.0265655   0.03671651 -0.02721134 -0.0116364
  0.03017285 -0.03030202  0.03052575 -0.01099537]
[('것', 0.610605182820184), ('별', 0.5105746389509589), ('점', 0.4896161970484222), ('부끄럼', 0.46506178731527686)] 

[-0.01958986  0.02647205  0.0265655   0.03671651 -0.02721134 -0.0116364
  0.03017285 -0.03030202  0.03052575 -0.01099537]
[('것', 0.610605182820184), ('별', 0.5105746389509589), ('점', 0.4896161970484222), ('부끄럼', 0.46506178731527686)]


Embedding Layer  
->Embedding Result랑 Neural Net과 연결하는 과정

In [ ]:
import torch
import torch.nn as nn

train_data = 'I like deep learning I like NLP I enjoy flying'

# 중복을 제거한 단어들의 집합인 단어 집합 생성.
word_set = set(train_data.split()) 

# 단어 집합의 각 단어에 고유한 정수 맵핑
vocab = {word: i+2 for i, word in enumerate(word_set)} 
vocab['unk'] = 0
vocab['pad'] = 1
print(vocab)

# 임베딩 테이블: (단어수, 임베딩 사이즈)
#Word2Vec, Glove, Random값으로 원래 설정되어있어야함
embedding_table = torch.FloatTensor([[ 0.0,  0.0,  0.0],
                                     [ 0.0,  0.0,  0.0],
                                     [ 0.1,  0.8,  0.3],
                                     [ 0.7,  0.8,  0.2],
                                     [ 0.1,  0.8,  0.7],
                                     [ 0.9,  0.2,  0.1],
                                     [ 0.1,  0.1,  0.9],
                                     [ 0.2,  0.1,  0.7],
                                     [ 0.3,  0.1,  0.1]])

# 입력 문장
input_snt = 'I like football'.split()

# 각 단어를 정수로 변환
idxes=[]

for word in input_snt:
  idx = vocab[word] if word in vocab else vocab['unk']
  idxes.append(idx)

idxes = torch.LongTensor(idxes)

# 입력 문장의임베딩가져오기: ['I', 'like', 'football (unk)']
lookup_result = embedding_table[idxes, :]
print(lookup_result)

# 임베딩 층 만들기
embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=3, padding_idx=1)
print(embedding_layer.weight)

{'flying': 2, 'I': 3, 'deep': 4, 'like': 5, 'NLP': 6, 'enjoy': 7, 'learning': 8, 'unk': 0, 'pad': 1}
tensor([[0.7000, 0.8000, 0.2000],
        [0.9000, 0.2000, 0.1000],
        [0.0000, 0.0000, 0.0000]])
Parameter containing:
tensor([[ 1.5768, -0.2788,  0.6780],
        [ 0.0000,  0.0000,  0.0000],
        [ 1.8775, -0.2953, -0.3648],
        [ 1.5040,  0.5887,  0.7606],
        [ 0.0074,  1.9403, -0.4993],
        [-1.8123,  1.2519, -0.9563],
        [ 0.7473, -0.3667, -0.0252],
        [ 0.7017, -1.3224, -0.1045],
        [-2.7745,  0.3770,  0.5290]], requires_grad=True)


In [ ]:
!pip install sentencepiece

In [ ]:
import sentencepiece as spm

input_file = "/gdrive/MyDrive/colab/text_rep/corpus.txt"
vocab_size = 500
model_name = 'subword_kor'
model_type = 'bpe'
user_defined_symbols = '[PAD],[UNK],[CLS],[SEP],[MASK],[UNK1],[UNK2],[UNK3],[UNK4],[UNK5]'

input_argument = '--input=%s --model_prefix=%s --vocab_size=%s --user_defined_symbols=%s --model_type=%s'
cmd = input_argument%(input_file, model_name, vocab_size,user_defined_symbols, model_type)

spm.SentencePieceTrainer.Train(cmd)

sp = spm.SentencePieceProcessor()
sp.Load("subword_kor.model")
wp = sp.EncodeAsPieces("하늘을 우러러")
id = sp.EncodeAsIds("하늘을 우러러")
print(wp)
print(id)

wp2snt= sp.DecodePieces(wp)
id2snt = sp.DecodeIds(id)
print(wp2snt)
print(id2snt)

print("Vocab. Size =",sp.GetPieceSize())
print(sp.IdToPiece(58))
print(sp.PieceToId('하'))

['▁', '하', '늘', '을', '▁우', '러', '러']
[174, 193, 356, 186, 80, 315, 315]
하늘을 우러러
하늘을 우러러
Vocab. Size = 500
▁강
193
